In [ ]:
!pip install -r requirements.txt

In [1]:
import boto3
from importlib import import_module
from dynaconf import Dynaconf
import os
import pandas as pd
from datetime import datetime

from src.judge import judge

In [26]:
judge_model = "sonnet-3"
model = 'nova-lite'
tones = ['casual', 'elaborate', 'emojify', 'improve', 'keypoints', 'professional', 'proofread', 'shorten', 'witty']

In [3]:
settings = Dynaconf(settings_files=["settings.toml"], env=judge_model, environments=True)

In [4]:
bedrock_client = boto3.client(service_name = 'bedrock-runtime', region_name = settings.region)

## Generic

In [5]:
# d = pd.read_csv('~/data/all.csv')

folder_path = os.path.expanduser('~/data')
files = sorted([f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))])
last_file = files[-2]

d = pd.read_csv(os.path.join(folder_path, last_file))

In [29]:
for tone in tones:
    print(f'''
    ---------------------
    {tone}
    ---------------------
    ''')
    dmt = d[(d.tone == tone) * (d.model == model)]
    if dmt.overall_score.isnull().any().item():
        queries, preds = dmt.input.values, dmt.output.values
        print(queries[0], preds[0])
        a = judge(settings.model, bedrock_client, queries, preds, tone)
        d.loc[(d.tone == tone) * (d.model == model), [a.lower() for a in a.columns]] = a.values


    ---------------------
    casual
    ---------------------
    

    ---------------------
    elaborate
    ---------------------
    

    ---------------------
    emojify
    ---------------------
    

    ---------------------
    improve
    ---------------------
    

    ---------------------
    keypoints
    ---------------------
    

    ---------------------
    professional
    ---------------------
    

    ---------------------
    proofread
    ---------------------
    

    ---------------------
    shorten
    ---------------------
    

    ---------------------
    witty
    ---------------------
    
I went to the store to buy some groceries. I embarked on a grand expedition to the store, where I procured an assortment of groceries, proving once again that I am a master of the modern-day grocery run.


In [34]:
t = datetime.now().timestamp()
d.to_csv(f'~/data/all-{t}.csv', index=False)

In [33]:
100 * (d.groupby(['model', 'tone'])['overall_score'].mean() - 1) / 2

model      tone        
haiku      casual          79.166667
           elaborate       90.025253
           emojify         89.750000
           improve         92.654639
           keypoints       86.734694
           professional    94.342105
           proofread       97.390110
           shorten         85.969388
           witty           68.956044
haiku-3-5  casual          98.878205
           elaborate       97.066327
           emojify         96.500000
           improve         88.690476
           keypoints       99.613402
           professional    93.382353
           proofread       94.047619
           shorten         97.938144
           witty           84.075342
nova-lite  casual          98.717949
           elaborate       92.346939
           emojify         86.750000
           improve         93.055556
           keypoints       94.716495
           professional    88.970588
           proofread       97.420635
           shorten         96.950172
           wit

In [32]:
100 * (d[d.model == model]['overall_score'].mean() - 1) / 2

np.float64(92.80082484230957)

## Rewrite Judge

In [ ]:
from src.judge import rewrite_judge

tones = ['casual', 'elaborate', 'emojify', 'improve', 'keypoints', 'professional', 'proofread', 'shorten', 'witty']
models = ['haiku', 'qwen', 'phi']

In [ ]:
folder_path = os.path.expanduser('~/data')
files = sorted([f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))])
last_file = files[-2]

d = pd.read_csv(os.path.join(folder_path, last_file))

for t in tones:
    for m in models:
        f = f'data/tones/{t.lower()}_{m}_rewrite.csv'
        if os.path.isfile(f):
            print(f'tone:{t}, model:{m} – Already done')
        else:
            print(f'tone:{t}, model:{m}')
    
            a = rewrite_judge(modelId, bedrock_client, queries, preds)
            print(a.rewrite_score.astype(int).mean())
            a.to_csv(f)